# 第三课：添加工具 (Tools) 能力

## 学习目标
1. 了解什么是 Tool（工具）
2. 学会创建自定义工具
3. 让 AI 能够调用工具

## 为什么需要工具？

大语言模型有一些固有的局限性：
- 无法获取实时信息（训练数据有截止日期）
- 数学计算可能出错
- 无法直接与外部系统交互

**工具**让 AI 能够：
- 🌤️ 获取实时信息（天气、新闻等）
- 🔢 执行精确计算
- 🔗 与外部系统交互（数据库、API等）

## 准备工作

In [ ]:
import os
from datetime import datetime
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

# 设置 API Key
os.environ["DASHSCOPE_API_KEY"] = "your_api_key_here"

def create_qwen_chat():
    api_key = os.getenv("DASHSCOPE_API_KEY")
    if not api_key or api_key == "your_api_key_here":
        raise ValueError("⚠️ 请设置你的 DASHSCOPE_API_KEY")
    
    return ChatOpenAI(
        model="qwen-plus",
        openai_api_key=api_key,
        openai_api_base="https://dashscope.aliyuncs.com/compatible-mode/v1",
        temperature=0,  # 使用工具时建议设为 0，让输出更确定
    )

print("✅ 准备完成")

---
## 1. 创建自定义工具

使用 `@tool` 装饰器是创建工具最简单的方式。

⚠️ **重要**：函数的 **docstring** 会被用作工具的描述，AI 会根据描述决定何时使用这个工具。

In [ ]:
from langchain_core.tools import tool

@tool
def get_current_time() -> str:
    """获取当前的日期和时间。当用户询问现在几点或今天日期时使用此工具。"""
    now = datetime.now()
    return now.strftime("%Y年%m月%d日 %H:%M:%S")

# 查看工具信息
print(f"工具名称: {get_current_time.name}")
print(f"工具描述: {get_current_time.description}")
print(f"参数: {get_current_time.args}")

In [ ]:
# 直接调用工具测试
result = get_current_time.invoke({})
print(f"当前时间: {result}")

In [ ]:
@tool
def calculate(expression: str) -> str:
    """
    计算数学表达式。支持加减乘除、幂运算等。
    
    Args:
        expression: 要计算的数学表达式，例如 "2 + 3 * 4" 或 "2 ** 10"
    
    Returns:
        计算结果
    """
    try:
        # 简单的安全检查
        allowed_chars = set("0123456789+-*/.() ")
        if not all(c in allowed_chars for c in expression):
            return "错误：表达式包含不允许的字符"
        
        result = eval(expression)
        return f"{expression} = {result}"
    except Exception as e:
        return f"计算错误：{str(e)}"

# 测试
print(calculate.invoke({"expression": "2 + 3 * 4"}))
print(calculate.invoke({"expression": "2 ** 10"}))

In [ ]:
@tool
def get_weather(city: str) -> str:
    """
    获取指定城市的天气信息。
    
    Args:
        city: 城市名称，例如 "北京"、"上海"
    
    Returns:
        天气信息
    """
    # 这是模拟数据，实际应用中应该调用真实的天气 API
    weather_data = {
        "北京": {"temp": 15, "condition": "晴", "humidity": 45},
        "上海": {"temp": 18, "condition": "多云", "humidity": 60},
        "广州": {"temp": 25, "condition": "小雨", "humidity": 80},
        "深圳": {"temp": 26, "condition": "阴", "humidity": 75},
    }
    
    if city in weather_data:
        data = weather_data[city]
        return f"{city}天气：{data['condition']}，温度 {data['temp']}°C，湿度 {data['humidity']}%"
    else:
        return f"抱歉，暂无 {city} 的天气数据。支持的城市：北京、上海、广州、深圳"

# 测试
print(get_weather.invoke({"city": "北京"}))
print(get_weather.invoke({"city": "成都"}))

---
## 2. 将工具绑定到模型

使用 `bind_tools()` 方法将工具绑定到模型，这样模型就知道有哪些工具可用。

In [ ]:
# 创建模型并绑定工具
model = create_qwen_chat()
tools = [get_current_time, calculate, get_weather]

# 绑定工具
model_with_tools = model.bind_tools(tools)

print("✅ 工具已绑定到模型")
print("\n可用工具:")
for t in tools:
    print(f"  - {t.name}: {t.description[:40]}...")

---
## 3. AI 决定是否使用工具

当用户提问时，AI 会判断是否需要使用工具：
- 如果需要，返回 `tool_calls`（工具调用请求）
- 如果不需要，直接返回回答

In [ ]:
# 测试：不需要工具的问题
response = model_with_tools.invoke("你好，介绍一下你自己")

print("问题: 你好，介绍一下你自己")
print(f"\n是否调用工具: {bool(response.tool_calls)}")
print(f"回复: {response.content}")

In [ ]:
# 测试：需要工具的问题
response = model_with_tools.invoke("现在几点了？")

print("问题: 现在几点了？")
print(f"\n是否调用工具: {bool(response.tool_calls)}")

if response.tool_calls:
    print("\n工具调用请求:")
    for call in response.tool_calls:
        print(f"  - 工具: {call['name']}")
        print(f"    参数: {call['args']}")
        print(f"    ID: {call['id']}")

In [ ]:
# 测试更多问题
questions = [
    "计算 15 * 28 + 100",
    "北京今天天气怎么样？",
    "Python 是什么？",  # 不需要工具
]

for q in questions:
    response = model_with_tools.invoke(q)
    print(f"\n问题: {q}")
    if response.tool_calls:
        print(f"  → 需要工具: {[c['name'] for c in response.tool_calls]}")
    else:
        print(f"  → 直接回答: {response.content[:50]}...")

---
## 4. 完整的工具调用流程

完整流程：
1. 用户提问
2. AI 判断是否需要工具
3. 如果需要，执行工具
4. 将工具结果返回给 AI
5. AI 生成最终回答

```
用户 → AI (判断) → 工具 → AI (总结) → 用户
```

In [ ]:
# 创建工具映射
tool_map = {
    "get_current_time": get_current_time,
    "calculate": calculate,
    "get_weather": get_weather,
}

def process_query(query: str) -> str:
    """
    处理用户查询的完整流程
    """
    print(f"\n{'='*50}")
    print(f"👤 用户: {query}")
    print(f"{'='*50}")
    
    # 第一步：发送问题给 AI
    messages = [HumanMessage(content=query)]
    response = model_with_tools.invoke(messages)
    
    # 第二步：检查是否需要调用工具
    if not response.tool_calls:
        print("\n[无需工具] AI 直接回答")
        return response.content
    
    # 第三步：执行工具调用
    print(f"\n[需要工具] AI 请求调用 {len(response.tool_calls)} 个工具")
    
    # 将 AI 的响应添加到消息列表
    messages.append(response)
    
    # 执行每个工具调用
    for tool_call in response.tool_calls:
        tool_name = tool_call["name"]
        tool_args = tool_call["args"]
        tool_id = tool_call["id"]
        
        print(f"\n  🔧 调用工具: {tool_name}")
        print(f"     参数: {tool_args}")
        
        # 执行工具
        tool = tool_map.get(tool_name)
        if tool:
            result = tool.invoke(tool_args)
            print(f"     结果: {result}")
        else:
            result = f"未知工具: {tool_name}"
        
        # 将工具结果添加到消息列表
        messages.append(ToolMessage(
            content=str(result),
            tool_call_id=tool_id
        ))
    
    # 第四步：让 AI 根据工具结果生成最终回答
    print("\n[生成回答] AI 根据工具结果生成最终回答")
    final_response = model_with_tools.invoke(messages)
    
    return final_response.content

In [ ]:
# 测试完整流程
answer = process_query("现在北京时间几点？")
print(f"\n🤖 最终回答: {answer}")

In [ ]:
# 测试计算
answer = process_query("帮我算一下，如果每月存 3000 元，一年能存多少？")
print(f"\n🤖 最终回答: {answer}")

In [ ]:
# 测试天气
answer = process_query("深圳和广州哪个城市今天更热？")
print(f"\n🤖 最终回答: {answer}")

In [ ]:
# 测试复杂问题（可能需要多个工具）
answer = process_query("告诉我现在的时间，以及北京的天气")
print(f"\n🤖 最终回答: {answer}")

---
## 5. 创建更复杂的工具

使用 `StructuredTool` 和 Pydantic 模型创建参数更复杂的工具。

In [ ]:
from langchain_core.tools import StructuredTool
from pydantic import BaseModel, Field

# 定义参数模型
class UnitConversionInput(BaseModel):
    """单位转换工具的输入参数"""
    value: float = Field(description="要转换的数值")
    from_unit: str = Field(description="原始单位，如 km, m, cm")
    to_unit: str = Field(description="目标单位，如 km, m, cm")

def unit_converter(value: float, from_unit: str, to_unit: str) -> str:
    """执行单位转换"""
    # 转换到基础单位（米）
    to_meter = {
        "km": 1000,
        "m": 1,
        "cm": 0.01,
        "mm": 0.001,
        "mile": 1609.34,
        "ft": 0.3048,
    }
    
    if from_unit not in to_meter or to_unit not in to_meter:
        return f"不支持的单位。支持: {list(to_meter.keys())}"
    
    meters = value * to_meter[from_unit]
    result = meters / to_meter[to_unit]
    
    return f"{value} {from_unit} = {result:.4f} {to_unit}"

# 创建结构化工具
converter_tool = StructuredTool.from_function(
    func=unit_converter,
    name="unit_converter",
    description="长度单位转换工具。支持 km, m, cm, mm, mile, ft 之间的转换。",
    args_schema=UnitConversionInput
)

print(f"工具名称: {converter_tool.name}")
print(f"工具描述: {converter_tool.description}")
print(f"参数结构: {converter_tool.args}")

In [ ]:
# 测试单位转换工具
print(converter_tool.invoke({"value": 5, "from_unit": "km", "to_unit": "mile"}))
print(converter_tool.invoke({"value": 100, "from_unit": "cm", "to_unit": "m"}))
print(converter_tool.invoke({"value": 1, "from_unit": "mile", "to_unit": "km"}))

---
## 6. 练习

尝试创建自己的工具：

In [ ]:
# 练习：创建一个 BMI 计算工具

@tool
def calculate_bmi(weight: float, height: float) -> str:
    """
    计算 BMI（身体质量指数）。
    
    Args:
        weight: 体重，单位：公斤
        height: 身高，单位：米
    
    Returns:
        BMI 值和健康建议
    """
    bmi = weight / (height ** 2)
    
    if bmi < 18.5:
        status = "偏瘦"
    elif bmi < 24:
        status = "正常"
    elif bmi < 28:
        status = "偏胖"
    else:
        status = "肥胖"
    
    return f"BMI = {bmi:.1f}，状态：{status}"

# 测试
print(calculate_bmi.invoke({"weight": 70, "height": 1.75}))
print(calculate_bmi.invoke({"weight": 80, "height": 1.70}))

In [ ]:
# 将 BMI 工具绑定到模型并测试
model_with_bmi = model.bind_tools([calculate_bmi, calculate])

# 更新工具映射
tool_map["calculate_bmi"] = calculate_bmi

# 临时替换模型
original_model = model_with_tools
model_with_tools = model_with_bmi

answer = process_query("我身高1.75米，体重70公斤，帮我算算BMI")
print(f"\n🤖 最终回答: {answer}")

# 恢复原模型
model_with_tools = original_model

---
## 📝 本课小结

| 概念 | 说明 |
|------|------|
| `@tool` 装饰器 | 将函数转换为工具，docstring 作为描述 |
| `bind_tools()` | 将工具绑定到模型 |
| `tool_calls` | AI 返回的工具调用请求 |
| `ToolMessage` | 将工具执行结果返回给 AI |
| `StructuredTool` | 创建参数更复杂的工具 |

### 工具调用流程

```
1. 用户提问
       ↓
2. AI 判断是否需要工具
       ↓
3. 执行工具，获取结果
       ↓
4. 将结果返回给 AI
       ↓
5. AI 生成最终回答
```

## ➡️ 下一课

下一课我们将使用 **LangGraph** 构建真正的智能体！LangGraph 会自动处理工具调用的循环，让代码更简洁。